In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
import re
from unidecode import unidecode
tqdm.pandas()

In [2]:
page = requests.get("https://thefangirlverdict.com/index/all-reviews/full-list-of-shows/")
soup = BeautifulSoup(page.content)

In [3]:
for p in soup.find_all("p", style="text-align:justify"):
    p.decompose()

In [4]:
names = []
urls = []
grades = []

for x in tqdm(range(1200)):
    
    element = soup.select_one("#post-7658 > div > p:nth-child(" + str(x) + ")")
    if element is None:
        continue
    
    kdrama = element.findChildren("a" , recursive=False)
    #print(kdrama)
    grade = element.findChildren("strong", recursive=False)
    #print(grade)
    
    if kdrama:
        names.append(kdrama[0].contents[0])
        urls.append(kdrama[0].get("href"))
        if grade:
            grades.append(grade[0].text)
        else:
            grades.append("NA")

100%|███████████████████████████████████████| 1200/1200 [14:12<00:00,  1.41it/s]


In [5]:
df = pd.DataFrame({"source": names, "url": urls, "rating": grades})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  688 non-null    object
 1   url     688 non-null    object
 2   rating  688 non-null    object
dtypes: object(3)
memory usage: 16.2+ KB


In [6]:
df.head()

,source,url,rating
0,here,http://thefangirlverdict.com/index/all-reviews...,NA
1,this post,https://thefangirlverdict.com/2021/01/12/dear-...,NA
2,1% of Anything [Something About 1%],http://thefangirlverdict.com/2017/04/29/flash-...,B+
3,9 Seconds: Eternal Time [Web Drama],https://thefangirlverdict.com/2022/02/11/flash...,B+
4,18 Again,https://thefangirlverdict.com/2021/01/08/revie...,A


In [ ]:
# df['rating'] = df['rating'].replace(['A++', 'A+', 'A', 'A-','B++','B+','B','B-','C++', 'C+', 'C', 'C-','D++','D+','D','D-'],['A1', 'A2', 'A3', 'A4','B1','B2','B3','B4','C1', 'C2', 'C3', 'C4','D1','D2','D3','D4'])

In [7]:
# handle NAs
df = df.replace('NA', np.NaN)
df.dropna(inplace = True)

In [8]:
df["year"] = df.apply(lambda row: re.search(r'\d+', row["url"]).group(), axis=1)
# df["source"] = df.apply(lambda row: unidecode(row["source"]), axis=1)
df.head()

,source,url,rating,year
2,1% of Anything [Something About 1%],http://thefangirlverdict.com/2017/04/29/flash-...,B+,2017
3,9 Seconds: Eternal Time [Web Drama],https://thefangirlverdict.com/2022/02/11/flash...,B+,2022
4,18 Again,https://thefangirlverdict.com/2021/01/08/revie...,A,2021
5,"1st Shop Of Coffee Prince, The [Coffee Prince]",http://thefangirlverdict.com/2019/01/03/review...,A,2019
6,20’s [Twenty Years Old],http://thefangirlverdict.com/2015/04/10/flash-...,B+,2015


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648 entries, 2 to 687
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  648 non-null    object
 1   url     648 non-null    object
 2   rating  648 non-null    object
 3   year    648 non-null    object
dtypes: object(4)
memory usage: 25.3+ KB


In [10]:
df.to_csv("df.csv", encoding='utf-8', index=False)